### **Loading Required Libraries**

In [1]:
import json
import re
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)

In [2]:
from transformers import __version__ as transformers_version
print(transformers_version)

4.45.1


In [3]:
from huggingface_hub import notebook_login
notebook_login()

# **Finetuning Llama3.2-3B Model on Salesforce/Xlam-function-calling Dataset**

### **Base Model Inference for JSON Generation**

In [4]:
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
prompt = 'Convert the following invalid json into valid json in single line: {"employee": "name": "Jane Smith", "designation": "Software Engineer"}'
messages = [
    {"role": "system", "content": "You are Qwen, a helpful assistant. Your task is to convert any input into valid json"},
    {"role": "user", "content": prompt}
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


{'role': 'assistant', 'content': '`{"employee": {"name": "Jane Smith", "designation": "Software Engineer"}}`'}


### **Loading the Base Model**

In [5]:
model_name = "meta-llama/Llama-3.2-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
if model.config.pad_token_id is None:
    model.config.pad_token_id = model.config.eos_token_id

### **Loading the Dataset**

In [7]:
dataset = load_dataset("Salesforce/xlam-function-calling-60k",split="train")
dataset

Dataset({
    features: ['answers', 'query', 'tools', 'id'],
    num_rows: 60000
})

In [8]:
dataset["query"][0]

'Where can I find live giveaways for beta access and games?'

In [9]:
dataset["answers"][0]

'[{"name": "live_giveaways_by_type", "arguments": {"type": "beta"}}, {"name": "live_giveaways_by_type", "arguments": {"type": "game"}}]'

### **Formatting the Dataset**

In [10]:
def format_llama_prompt(example):
    """Format the prompt according to LLaMA 3.2-3B chat template."""
    prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n{example['query']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n{example['answers']}"
    return {"prompt": prompt}

dataset = dataset.map(format_llama_prompt)

In [11]:
dataset

Dataset({
    features: ['answers', 'query', 'tools', 'id', 'prompt'],
    num_rows: 60000
})

In [13]:
dataset['prompt'][1]

'<|begin_of_text|><|start_header_id|>user<|end_header_id|>\nI need to understand the details of the Ethereum blockchain for my cryptocurrency project. Can you fetch the details for \'ethereum\'?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n[{"name": "web_chain_details", "arguments": {"chain_slug": "ethereum"}}]'

### **Tokenization of Dataset**

In [14]:
def tokenize_function(examples):
    return tokenizer(examples["prompt"], truncation=True, padding="max_length", max_length=1024)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [15]:
tokenized_dataset

Dataset({
    features: ['answers', 'query', 'tools', 'id', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 60000
})

In [16]:
dataset = tokenized_dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]
dataset

DatasetDict({
    train: Dataset({
        features: ['answers', 'query', 'tools', 'id', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 48000
    })
    test: Dataset({
        features: ['answers', 'query', 'tools', 'id', 'prompt', 'input_ids', 'attention_mask'],
        num_rows: 12000
    })
})

### **Apply Lora**

In [17]:
import bitsandbytes as bnb

# Find all linear module names
def find_all_linear_names(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):  # Standard torch.nn.Linear for fine-tuning
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
        if 'lm_head' in lora_module_names:  # Exclude lm_head from fine-tuning
            lora_module_names.remove('lm_head')
    return list(lora_module_names)

modules = find_all_linear_names(model)
print("LoRA Modules to Fine-Tune:", modules)

from peft import LoraConfig, get_peft_model
# LoRA configuration for fine-tuning
lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=['o_proj', 'q_proj', 'up_proj', 'v_proj', 'k_proj', 'down_proj', 'gate_proj'],  # Target LoRA modules
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"  # Task type set to Causal Language Modeling
)

# Wrap the model with LoRA for fine-tuning
model = get_peft_model(model, lora_config)

LoRA Modules to Fine-Tune: ['v_proj', 'o_proj', 'k_proj', 'q_proj', 'gate_proj', 'down_proj', 'up_proj']


In [18]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | Total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 97255424 | Total: 3310005248 | Percentage: 2.9382%


### **Setting Up Training Arguments and Training for SFTTrainer**

In [19]:
import transformers
from trl import SFTTrainer
import torch

tokenizer.pad_token = tokenizer.eos_token

torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,  
    train_dataset=train_data,  
    eval_dataset=test_data,  
    dataset_text_field="prompt",  
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,  
        gradient_accumulation_steps=4,  
        num_train_epochs=50, 
        warmup_steps=10,  
        max_steps=1500,  
        learning_rate=2e-4,  
        logging_steps=5,  
        output_dir="outputs_llama3.2_1b_sft", 
        optim="adamw_hf",  
        save_strategy="epoch",  
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),  # Collator for causal language modeling
)

# Run training
trainer.train()

/home/geek/.local/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/geek/anaconda3/envs/llama/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/home/geek/anaconda3/envs/llama/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
/home/geek/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWa

  0%|          | 0/1500 [00:00<?, ?it/s]

{'loss': 3.3186, 'grad_norm': 2.166571617126465, 'learning_rate': 0.0001, 'epoch': 0.0}
{'loss': 2.5665, 'grad_norm': 0.9053242206573486, 'learning_rate': 0.0002, 'epoch': 0.0}
{'loss': 1.7173, 'grad_norm': 1.0338925123214722, 'learning_rate': 0.00019932885906040267, 'epoch': 0.0}
{'loss': 1.4843, 'grad_norm': 0.7942467331886292, 'learning_rate': 0.0001986577181208054, 'epoch': 0.0}
{'loss': 1.4548, 'grad_norm': 0.7946552634239197, 'learning_rate': 0.00019798657718120806, 'epoch': 0.0}
{'loss': 1.2094, 'grad_norm': 0.9228646755218506, 'learning_rate': 0.00019731543624161075, 'epoch': 0.0}
{'loss': 1.3435, 'grad_norm': 1.3663142919540405, 'learning_rate': 0.00019664429530201342, 'epoch': 0.0}
{'loss': 1.2828, 'grad_norm': 2.510617971420288, 'learning_rate': 0.00019597315436241613, 'epoch': 0.0}
{'loss': 1.2173, 'grad_norm': 0.8956913352012634, 'learning_rate': 0.0001953020134228188, 'epoch': 0.0}
{'loss': 1.0852, 'grad_norm': 0.8107909560203552, 'learning_rate': 0.0001946308724832215, '

TrainOutput(global_step=1500, training_loss=0.8732972733179728, metrics={'train_runtime': 2961.0576, 'train_samples_per_second': 2.026, 'train_steps_per_second': 0.507, 'total_flos': 1.07495527809024e+17, 'train_loss': 0.8732972733179728, 'epoch': 0.125})

### **Merge and Save the Finetuned Model**

In [20]:
new_model = "llama-3.2-3b-finetune-xlam-function-calling-60k"
trainer.model.save_pretrained(new_model)
model_id = "meta-llama/Llama-3.2-3B-Instruct"
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

merged_model.save_pretrained("merged_model_sft3.2_3b",safe_serialization=True)
tokenizer.save_pretrained("merged_model_sft3.2_3b")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### **Loading the Saved Model**

In [21]:
finetuned_model = AutoModelForCausalLM.from_pretrained("merged_model_sft3.2_3b", torch_dtype=torch.float16, device_map="auto")

finetuned_tokenizer = AutoTokenizer.from_pretrained("merged_model_sft3.2_3b")
finetuned_tokenizer.pad_token = finetuned_tokenizer.eos_token
finetuned_tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


### **Push the model to Huggingface**

In [ ]:
repo_name = "Amin-01/Llama-3.2-3b-finetuned-for-json-generation"

finetuned_model.push_to_hub(repo_name)
finetuned_tokenizer.push_to_hub(repo_name)

/home/geek/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/modeling_utils.py:2670: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/abdulmannan-01/Llama-3.2-3b-finetuned-for-json-generation/commit/d05c6dbd9445283bbb3be184393f94b5dc3f6ccc', commit_message='Upload tokenizer', commit_description='', oid='d05c6dbd9445283bbb3be184393f94b5dc3f6ccc', pr_url=None, pr_revision=None, pr_num=None)

# **Inference Using Finetuned Huggingface Model**

### **Load Model From HuggingFace**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json

finetuned_model = AutoModelForCausalLM.from_pretrained(
    "Amin-01/Llama-3.2-3b-finetuned-for-json-generation",
    torch_dtype=torch.float16,
    device_map="auto"
)
finetuned_tokenizer = AutoTokenizer.from_pretrained("Amin-01/Llama-3.2-3b-finetuned-for-json-generation")
finetuned_tokenizer.pad_token = finetuned_tokenizer.eos_token
finetuned_tokenizer.padding_side = 'left'

In [2]:
# Add special tokens if not already present
special_tokens = {
    'additional_special_tokens': [
        '<|begin_of_text|>',
        '<|start_header_id|>',
        '<|end_header_id|>',
        '<|eot_id|>',
        '<|endoftext|>'
    ]
}
num_added_toks = finetuned_tokenizer.add_special_tokens(special_tokens)
if num_added_toks > 0:
    finetuned_model.resize_token_embeddings(len(finetuned_tokenizer))

In [3]:
def prepare_input(query):
    # Construct the prompt according to the Llama3.2-1B chat template
    prompt = (
        "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n"
        + query
        + "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
    )
    # Tokenize the prompt
    inputs = finetuned_tokenizer(prompt, return_tensors='pt').to('cuda')
    return inputs

In [4]:
def extract_assistant_response(output_text):
    assistant_header = "<|start_header_id|>assistant<|end_header_id|>\n"
    start_idx = output_text.find(assistant_header)
    if start_idx == -1:
        print("Assistant header not found in the output.")
        return ''
    start_idx += len(assistant_header)
    
    # Extract the assistant's content
    assistant_content = output_text[start_idx:].strip()
    
    # Stop at the next occurrence of the assistant header to avoid repeated outputs
    split_token = '<|start_header_id|>assistant<|end_header_id|>'
    end_idx = assistant_content.find(split_token)
    if end_idx != -1:
        assistant_content = assistant_content[:end_idx].strip()
    
    # Remove any trailing special tokens
    special_tokens = [
        '<|eot_id|>',
        '<|endoftext|>',
        '<|end_of_text|>',
        '<|end_header_id|>',
        '<|start_header_id|>',
        '<|begin_of_text|>'
    ]
    
    # Remove special tokens from the end of assistant_content
    while True:
        for token in special_tokens:
            if assistant_content.endswith(token):
                assistant_content = assistant_content[:-len(token)].strip()
                break
        else:
            break
    
    return assistant_content


In [37]:
def generate_response(model, inputs):
    eot_token_id = finetuned_tokenizer.convert_tokens_to_ids('<|eot_id|>')
    
    # Generate the output
    output = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        pad_token_id=finetuned_tokenizer.eos_token_id,
        eos_token_id=eot_token_id, 
        early_stopping=True
    )
    output_text = finetuned_tokenizer.decode(output[0], skip_special_tokens=False)
    # Uncomment the line below to see the full generated text including special tokens
    #print('Model Output:', output_text)
    assistant_response = extract_assistant_response(output_text)
    return assistant_response


In [23]:
def isJsonString(input_string):
    try:
        json.loads(input_string)
        return True
    except ValueError:
        return False

In [26]:
query1 = "Fetch auto-complete results for 'winter coat' from Kohls and 'kitchen appliance' from Wayfair."
inputs1 = prepare_input(query1)
response1 = generate_response(finetuned_model, inputs1)

print("Generated JSON response:", response1)

Generated JSON response: [{"name": "auto_complete", "arguments": {"query": "winter coat", "store": "Kohls"}}, {"name": "auto_complete", "arguments": {"query": "kitchen appliance", "store": "Wayfair"}}]


In [27]:
isJsonString(response1)

True

### **Testing on Different Text Queries**

In [28]:
query2 = "'ETH/BTC' using a 1h interval and a time period of 14?"
inputs2 = prepare_input(query2)
response2 = generate_response(finetuned_model, inputs2)

print("Generated JSON response:", response2)

Generated JSON response: [{"name": "binance_ticker", "arguments": {"symbol": "ETH/BTC", "interval": "1h", "time_period": 14}}]


In [29]:
isJsonString(response2)

True

In [33]:
query3 = "I need to understand the details of the Ethereum blockchain for my cryptocurrency project. Can you fetch the details for 'ethereum'?"
inputs3 = prepare_input(query3)
response3 = generate_response(finetuned_model, inputs3)

print("Generated JSON response:", response3)

Generated JSON response: [{"name": "get_specific_chain", "arguments": {"chain": "ethereum"}}]


In [34]:
isJsonString(response3)

True

In [46]:
query4 = "I need a new password. Can you generate one for me?"
inputs4 = prepare_input(query4)
response4 = generate_response(finetuned_model, inputs4)

print("Generated JSON response:", response4)

Generated JSON response: [{"name": "generate_password", "arguments": {"length": 12}}]


In [47]:
isJsonString(response4)

True

### **Testing the Valid JSON Generation by providing non-valid JSONs and lengthy text queries**

In [48]:
query5 = '{"id": 1, "name": "John Doe", "age": 29, "is_student": false,}'
inputs5 = prepare_input(query5)
response5 = generate_response(finetuned_model, inputs5)

print("Generated JSON response:", response5)

Generated JSON response: [{"name": "getuserbyname", "arguments": {"username": "JohnDoe"}}]


In [49]:
isJsonString(response5)

True

In [50]:
query6 = '{"fruits": ["apple", "banana", "orange", "quantity": 3}'
inputs6 = prepare_input(query6)
response6 = generate_response(finetuned_model, inputs6)

print("Generated JSON response:", response6)

Generated JSON response: [{"name": "parse_fruit", "arguments": {"fruits": "apple,banana,orange", "quantity": 3}}]


In [51]:
isJsonString(response6)

True

In [52]:
query7 = '{"employee": "name": "Jane Smith", "designation": "Software Engineer"}'
inputs7 = prepare_input(query7)
response7 = generate_response(finetuned_model, inputs7)

print("Generated JSON response:", response7)

Generated JSON response: [{"name": "get_employee_details", "arguments": {"employee": "name: Jane Smith, designation: Software Engineer"}}]


In [53]:
isJsonString(response7)

True

In [54]:
query8 = '{"status": "success" "data": null}'
inputs8 = prepare_input(query8)
response8 = generate_response(finetuned_model, inputs8)

print("Generated JSON response:", response8)

Generated JSON response: [{"name": "getdata", "arguments": {"status": "success"}}]


In [55]:
isJsonString(response8)

True

In [57]:
query9 = '{"products": [{"id": 1, "name": "Laptop", "price": 999.99} {"id": 2, "name": "Mouse", "price": 19.99}]}'
inputs9 = prepare_input(query9)
response9 = generate_response(finetuned_model, inputs9)

print("Generated JSON response:", response9)

Generated JSON response: [{"name": "get_products", "arguments": {"products": [{"id": 1, "name": "Laptop", "price": 999.99}, {"id": 2, "name": "Mouse", "price": 19.99}]}}]


In [58]:
isJsonString(response9)

True

In [59]:
query10 = 'John Doe is a 34-year-old software engineer living in San Francisco. He works for a tech startup called Tech Innovators, where he leads the AI research team. John graduated from Stanford University with a degree in Computer Science and has over 10 years of experience in the field of artificial intelligence. '
inputs10 = prepare_input(query10)
response10 = generate_response(finetuned_model, inputs10)

print("Generated JSON response:", response10)

Generated JSON response: [{"name": "getprofile", "arguments": {"name": "John Doe", "age": 34, "occupation": "Software Engineer", "city": "San Francisco", "company": "Tech Innovators", "university": "Stanford University", "years_of_experience": 10, "area_of_expertise": "Artificial Intelligence"}}]


In [60]:
isJsonString(response10)

True

In [62]:
query11 = "John's current project involves building an AI system that helps businesses optimize their operations using predictive analytics. He is passionate about leveraging machine learning to solve complex business problems. Outside of work, John enjoys hiking, photography, and volunteering at local community events. He is also a mentor for aspiring software developers and often participates in hackathons as a judge."
inputs11 = prepare_input(query11)
response11 = generate_response(finetuned_model, inputs11)

print("Generated JSON response:", response11)

Generated JSON response: [{"name": "get_project_info", "arguments": {"project_name": "Building an AI system for predictive analytics in business operations"}}, {"name": "get_project_info", "arguments": {"project_name": "Building an AI system for predictive analytics in business operations"}}, {"name": "get_project_info", "arguments": {"project_name": "Building an AI system for predictive analytics in business operations"}}, {"name": "get_project_info", "arguments": {"project_name": "Building an AI system for predictive analytics in business operations"}}]


In [63]:
isJsonString(response11)

True

In [64]:
query12 = "John is married to Jane Doe, who works as a graphic designer. They have two children: Emily, aged 5, and Michael, aged 3. As a family, they enjoy weekend trips to national parks and spending time at the beach."
inputs12 = prepare_input(query12)
response12 = generate_response(finetuned_model, inputs12)

print("Generated JSON response:", response12)

Generated JSON response: [{"name": "family_profile", "arguments": {"first_name": "John", "last_name": "Doe", " occupation": "Graphic Designer", "children": ["Emily", "Michael"], "family_interests": ["weekend trips", "national parks", "beach"]}}]


In [65]:
isJsonString(response12)

True

In [66]:
query13 = "John's professional expertise includes programming languages such as Python, Java, and C++. He is proficient in frameworks like TensorFlow and PyTorch, and he is familiar with cloud services such as AWS and GCP. He regularly publishes articles on AI and machine learning topics and has been a keynote speaker at various tech conferences."
inputs13 = prepare_input(query13)
response13 = generate_response(finetuned_model, inputs13)

print("Generated JSON response:", response13)

Generated JSON response: [{"name": "get_the_expertise", "arguments": {"name": "John", "language": "Python", "framework": "TensorFlow", "cloud": "AWS", "topic": "AI", "experience": "keynote speaker"}}]


In [67]:
isJsonString(response13)

True

In [68]:
query14 = "Dan, Natalie and Sean went running together. The three of them are runners. Natalie is 45 years old while Both Dan and Sean are approaching 47. Natalie is working as an engineer in a tech company with Dan, they met there. Sean has a daughter who likes swimming, so he also goes swimming with her. Her name is Alice. Alice likes eating a pizza. Her best friend, Sara, is about the same age and enjoys icecream the most."
inputs14 = prepare_input(query14)
response14 = generate_response(finetuned_model, inputs14)

print("Generated JSON response:", response14)

Generated JSON response: [{"name": "analyze_personality", "arguments": {"name": "Natalie"}}, {"name": "analyze_personality", "arguments": {"name": "Dan"}}, {"name": "analyze_personality", "arguments": {"name": "Sean"}}, {"name": "analyze_personality", "arguments": {"name": "Alice"}}, {"name": "analyze_personality", "arguments": {"name": "Sara"}}]


In [69]:
isJsonString(response14)

True

In [74]:
query15 = "Once upon a time in a small village, there were ten unique individuals. There was Amelia, the talented artist, always seen with a sketchpad in her hand. Next was Benjamin, the curious scientist, constantly conducting experiments in his backyard. Clara, the kind-hearted nurse, spent her days tending to the ill and the injured. Darren, the mischievous trickster, always had a prank up his sleeve. Eliza, the aspiring chef, could be found in her kitchen, concocting delicious recipes. Fred, the wise old man, spent his time under his favorite tree, sharing stories with anyone who would listen. Grace, the passionate teacher, inspired her students with her energetic lessons. Harry, the adventurous explorer, was always venturing into the unknown, seeking new discoveries. Isabella, the gentle gardener, nurtured her magnificent botanical creations. Lastly, Jackson, the compassionate volunteer, dedicated his time to helping those less fortunate. These ten individuals, though different in their pursuits, shared a common love for their village and one another. Their unique talents and selflessness came together, creating a harmonious community that thrived, celebrated, and most importantly, lived happily ever after."
inputs15 = prepare_input(query15)
response15 = generate_response(finetuned_model, inputs15)

print("Generated JSON response:", response15)

Generated JSON response: [{"name": "get_all_characters", "arguments": {"text": "Once upon a time in a small village, there were ten unique individuals. There was Amelia, the talented artist, always seen with a sketchpad in her hand. Next was Benjamin, the curious scientist, constantly conducting experiments in his backyard. Clara, the kind-hearted nurse, spent her days tending to the ill and the injured. Darren, the mischievous trickster, always had a prank up his sleeve. Eliza, the aspiring chef, could be found in her kitchen, concocting delicious recipes. Fred, the wise old man, spent his time under his favorite tree, sharing stories with anyone who would listen. Grace, the passionate teacher, inspired her students with her energetic lessons. Harry, the adventurous explorer, was always venturing into the unknown, seeking new discoveries. Isabella, the gentle gardener, nurtured her magnificent botanical creations. Lastly, Jackson, the compassionate volunteer, dedicated his time to hel

In [75]:
isJsonString(response15)

True

In [76]:
query16 = '{"contact": {"phone": 123-456-7890, "email": "example@example.com"}, "preferences": {"contact_time": "morning", "newsletter_subscribed": true}}'
inputs16 = prepare_input(query16)
response16 = generate_response(finetuned_model, inputs16)

print("Generated JSON response:", response16)

Generated JSON response: [{"name": "get_contact_info", "arguments": {"phone": "123-456-7890", "email": "example@example.com"}}, {"name": "get_preferences", "arguments": {"contact_time": "morning", "newsletter_subscribed": true}}]


In [77]:
isJsonString(response16)

True

In [78]:
query17 = '{"books": [{"title": "The Catcher in the Rye", "author": "J.D. Salinger"}, {"title": "To Kill a Mockingbird", "author": "Harper Lee"}'
inputs17 = prepare_input(query17)
response17 = generate_response(finetuned_model, inputs17)

print("Generated JSON response:", response17)

Generated JSON response: [{"name": "get_all_books", "arguments": {"books": [{"title": "The Catcher in the Rye", "author": "J.D. Salinger"}, {"title": "To Kill a Mockingbird", "author": "Harper Lee"}]}}]


In [79]:
isJsonString(response17)

True

In [94]:
query18 = '{"sports_league": {"name": "National Basketball League", "teams": [{"name": "Lakers", "city": "Los Angeles", "players": [{"name": "LeBron James", "position": "SF"}, {"name": "Anthony Davis", "position": "PF"}]}, {"name": "Warriors", "city": "San Francisco", "players": [{"name": "Stephen Curry", "position": "PG"}, {"name": "Klay Thompson", "position": "SG"}]}], "sponsors": [{"company": "Nike", "amount": 2000000}, {"company": "Adidas", "amount": 1500000}]}}'
inputs18 = prepare_input(query18)
response18 = generate_response(finetuned_model, inputs18)

print("Generated JSON response:", response18)

Generated JSON response: [{"name": "sports_league", "arguments": {"sports_league": "National Basketball League", "teams": [{"name": "Lakers", "city": "Los Angeles", "players": [{"name": "LeBron James", "position": "SF"}, {"name": "Anthony Davis", "position": "PF"}]}, {"name": "Warriors", "city": "San Francisco", "players": [{"name": "Stephen Curry", "position": "PG"}, {"name": "Klay Thompson", "position": "SG"}]}], "sponsors": [{"company": "Nike", "amount": 2000000}, {"company": "Adidas", "amount": 1500000}]}}]


In [95]:
isJsonString(response18)

True

In [97]:
query19 = '{"research_lab": {"name": "AI Institute", "location": {"city": "Cambridge", "state": "MA"}, "projects": [{"name": "AI for Healthcare"}, {"name": "Autonomous Vehicles"]}}'
inputs19 = prepare_input(query19)
response19 = generate_response(finetuned_model, inputs19)

print("Generated JSON response:", response19)

Generated JSON response: [{"name": "getresearchlab", "arguments": {"name": "AI Institute", "location": {"city": "Cambridge", "state": "MA"}, "projects": [{"name": "AI for Healthcare"}, {"name": "Autonomous Vehicles"}]}}]


In [98]:
isJsonString(response19)

True

In [100]:
query20 = '{"tech_conference": {"name": "Future of AI", "date": "2024-09-15", "location": {"city": "San Francisco", "state": "CA"}, "speakers": [{"name": "Andrew Ng"}, {"name": "Geoffrey Hinton"]}}}'
inputs20 = prepare_input(query20)
response20 = generate_response(finetuned_model, inputs20)

print("Generated JSON response:", response20)

Generated JSON response: [{"name": "tech_conference", "arguments": {"name": "Future of AI", "date": "2024-09-15", "location": {"city": "San Francisco", "state": "CA"}, "speakers": [{"name": "Andrew Ng"}, {"name": "Geoffrey Hinton"}]}}]


In [101]:
isJsonString(response20)

True